In [1]:
import numpy as np
import pandas as pd
from skimage import transform
from keras.models import Model, Sequential, load_model
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D
from keras import losses, optimizers, callbacks
from keras.applications import vgg16, vgg19, inception_v3
import xgboost as xgb
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_validate, train_test_split
from scipy import fftpack
import matplotlib.pyplot as plt
%matplotlib inline
RANDOM_SEED = 43
np.random.seed(RANDOM_SEED)

Using TensorFlow backend.
/home/can/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Create Dataset

In [2]:
# Train
output_shape = (139, 139)
df = pd.read_json('./data/train.json')
X_train, y_train = [], []
for im_band1, im_band2, label in zip(df['band_1'], df['band_2'], df['is_iceberg']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)  
    im_band1_fft = fftpack.fftshift(np.log(np.abs(fftpack.fft2(im_band1))))
    im_bands_avg = (im_band1 + im_band2) / 2.0
#     # Preprocess - resize
#     im_band1 = transform.resize(im_band1, output_shape=output_shape)
#     im_band2 = transform.resize(im_band2, output_shape=output_shape)
#     im_bands_avg = transform.resize(im_bands_avg, output_shape=output_shape)
    # Preprocess - zero mean
    im_band1 -= np.mean(im_band1)
    im_band2 -= np.mean(im_band2)
    im_bands_avg -= np.mean(im_bands_avg)
    # Preprocess - normalize
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)
    im_bands_avg /= np.std(im_bands_avg)
    # Concatenate
    im = np.concatenate([im_band1, im_band2, im_bands_avg], axis=2)
    X_train.append(im)
    y_train.append(label)    
X_train = np.array(X_train)
y_train = np.array(y_train)
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape

X_train.shape: (1604, 75, 75, 3)
y_train.shape: (1604,)


# Train - Val Split

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=RANDOM_SEED)
print 'X_train.shape:', X_train.shape
print 'X_val.shape:', X_val.shape
print 'y_train.shape:', y_train.shape
print 'y_val.shape:', y_val.shape

X_train.shape: (1283, 75, 75, 3)
X_val.shape: (321, 75, 75, 3)
y_train.shape: (1283,)
y_val.shape: (321,)


# Data Augmentation

In [4]:
def bypass(x):
    return x

def h_flip(x):
    return x[:, :, ::-1, :]

def v_flip(x):
    return x[:, ::-1, :, :]

def hv_flip(x):
    return h_flip(v_flip(x))

def rot90(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 90), axis=0) for im in x], axis=0)

def rot180(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 180), axis=0) for im in x], axis=0)

def rot270(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 270), axis=0) for im in x], axis=0)

def rot45(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 45, mode='reflect'), axis=0) for im in x], axis=0)

def rot135(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 135, mode='reflect'), axis=0) for im in x], axis=0)

def rot315(x):
    return np.concatenate([np.expand_dims(transform.rotate(im, 315, mode='reflect'), axis=0) for im in x], axis=0)

aug_funcs = [bypass, 
             h_flip, v_flip, hv_flip,
             rot90, rot180, rot270]

In [5]:
# Train
X_train = np.concatenate([func(X_train) for func in aug_funcs], axis=0)
y_train = np.concatenate([y_train] * len(aug_funcs))

# Validation
X_val = np.concatenate([func(X_val) for func in aug_funcs], axis=0)
y_val = np.concatenate([y_val] * len(aug_funcs))

# 
print 'X_train.shape:', X_train.shape
print 'y_train.shape:', y_train.shape
print 'X_val.shape:', X_val.shape
print 'y_val.shape:', y_val.shape

X_train.shape: (8981, 75, 75, 3)
y_train.shape: (8981,)
X_val.shape: (2247, 75, 75, 3)
y_val.shape: (2247,)


# Training

In [6]:
# Make model
INPUT_SHAPE = (75, 75, 3)
base_model = vgg16.VGG16(include_top=False, input_shape=INPUT_SHAPE)
x = base_model.layers[-5].output
x = GlobalAveragePooling2D()(x)
# x = GlobalMaxPooling2D()(x)
# x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(inputs=base_model.input, outputs=x)

In [7]:
# -- Top Model Train --
for layer in  base_model.layers:
    layer.trainable = False

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.Adam()
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
MODEL_PATH = './models/model3.h5'
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.33, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 7s - loss: 0.4793 - acc: 0.7602 - val_loss: 0.3513 - val_acc: 0.8296
Epoch 2/200
8981/8981 [==============================] - 7s - loss: 0.3581 - acc: 0.8224 - val_loss: 0.3120 - val_acc: 0.8482
Epoch 3/200
8981/8981 [==============================] - 6s - loss: 0.3386 - acc: 0.8379 - val_loss: 0.2993 - val_acc: 0.8594
Epoch 4/200
8981/8981 [==============================] - 6s - loss: 0.3082 - acc: 0.8542 - val_loss: 0.2865 - val_acc: 0.8629
Epoch 5/200
8981/8981 [==============================] - 7s - loss: 0.2984 - acc: 0.8625 - val_loss: 0.2990 - val_acc: 0.8558
Epoch 6/200
8981/8981 [==============================] - 6s - loss: 0.2864 - acc: 0.8694 - val_loss: 0.2850 - val_acc: 0.8700
Epoch 7/200
8981/8981 [==============================] - 6s - loss: 0.2794 - acc: 0.8715 - val_loss: 0.2932 - val_acc: 0.8687
Epoch 8/200
8960/8981 [============================>.] - ETA: 0s - los

In [8]:
# Load the model
model = load_model(MODEL_PATH)

# -- Entire Model Train --
for layer in model.layers:
    layer.trainable = True

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=6, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 15s - loss: 0.2153 - acc: 0.9058 - val_loss: 0.2749 - val_acc: 0.8856
Epoch 2/200
8981/8981 [==============================] - 15s - loss: 0.2015 - acc: 0.9139 - val_loss: 0.2821 - val_acc: 0.8874
Epoch 3/200
8981/8981 [==============================] - 15s - loss: 0.1939 - acc: 0.9193 - val_loss: 0.2722 - val_acc: 0.8896
Epoch 4/200
8981/8981 [==============================] - 15s - loss: 0.1873 - acc: 0.9229 - val_loss: 0.2727 - val_acc: 0.8883
Epoch 5/200
8981/8981 [==============================] - 15s - loss: 0.1798 - acc: 0.9231 - val_loss: 0.2705 - val_acc: 0.8945
Epoch 6/200
8981/8981 [==============================] - 15s - loss: 0.1725 - acc: 0.9261 - val_loss: 0.2632 - val_acc: 0.8941
Epoch 7/200
8981/8981 [==============================] - 15s - loss: 0.1695 - acc: 0.9304 - val_loss: 0.2754 - val_acc: 0.8927
Epoch 8/200
8960/8981 [============================>.] - ETA: 0

In [9]:
# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-5)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=6, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2955 - acc: 0.8591 - val_loss: 0.2587 - val_acc: 0.8763
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.3002 - acc: 0.8595 - val_loss: 0.2606 - val_acc: 0.8763
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2952 - acc: 0.8625 - val_loss: 0.2582 - val_acc: 0.8772
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2966 - acc: 0.8574 - val_loss: 0.2593 - val_acc: 0.8727
Epoch 5/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2939 - acc: 0.8606Epoch 00004: val_loss did not improve

Epoch 00004: reducing learning rate to 4.99999987369e-06.
8981/8981 [==============================] - 10s - loss: 0.2944 - acc: 0.8604 - val_loss: 0.2582 - val_acc: 0.8736
Epoch 6/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2980 - acc: 0.8594Epoch 00005: val_loss did not improve

Epoch 0

In [10]:
# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-5)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=5, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2920 - acc: 0.8603 - val_loss: 0.2567 - val_acc: 0.8723
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2953 - acc: 0.8609 - val_loss: 0.2577 - val_acc: 0.8749
Epoch 3/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2916 - acc: 0.8641Epoch 00002: val_loss did not improve

Epoch 00002: reducing learning rate to 4.99999987369e-06.
8981/8981 [==============================] - 10s - loss: 0.2923 - acc: 0.8636 - val_loss: 0.2572 - val_acc: 0.8741
Epoch 4/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2974 - acc: 0.8636Epoch 00003: val_loss did not improve

Epoch 00003: reducing learning rate to 2.49999993684e-06.
8981/8981 [==============================] - 10s - loss: 0.2977 - acc: 0.8633 - val_loss: 0.2588 - val_acc: 0.8741
Epoch 5/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2944 - acc: 0

In [11]:
# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=5e-5)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=6, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2982 - acc: 0.8560 - val_loss: 0.2756 - val_acc: 0.8660
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2979 - acc: 0.8576 - val_loss: 0.2527 - val_acc: 0.8714
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2938 - acc: 0.8585 - val_loss: 0.2697 - val_acc: 0.8736
Epoch 4/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2952 - acc: 0.8598Epoch 00003: val_loss did not improve

Epoch 00003: reducing learning rate to 2.49999993684e-05.
8981/8981 [==============================] - 10s - loss: 0.2951 - acc: 0.8596 - val_loss: 0.2626 - val_acc: 0.8723
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2862 - acc: 0.8679 - val_loss: 0.2506 - val_acc: 0.8781
Epoch 6/200
8981/8981 [==============================] - 10s - loss: 0.2890 - acc: 0.8617 - val_loss: 0.2514 - val_acc: 0.8798
Epoch 7/200


In [12]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=5e-5)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=4, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2860 - acc: 0.8655 - val_loss: 0.2492 - val_acc: 0.8772
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2896 - acc: 0.8628 - val_loss: 0.2516 - val_acc: 0.8741
Epoch 3/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2877 - acc: 0.8628Epoch 00002: val_loss did not improve

Epoch 00002: reducing learning rate to 2.49999993684e-05.
8981/8981 [==============================] - 10s - loss: 0.2882 - acc: 0.8626 - val_loss: 0.2560 - val_acc: 0.8772
Epoch 4/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2874 - acc: 0.8673Epoch 00003: val_loss did not improve

Epoch 00003: reducing learning rate to 1.24999996842e-05.
8981/8981 [==============================] - 10s - loss: 0.2875 - acc: 0.8675 - val_loss: 0.2498 - val_acc: 0.8807
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2824 - acc: 0.869

In [13]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=9e-5)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2924 - acc: 0.8599 - val_loss: 0.2462 - val_acc: 0.8798
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2889 - acc: 0.8625 - val_loss: 0.2637 - val_acc: 0.8763
Epoch 3/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2908 - acc: 0.8654- ETA: 5s - ETA: 4s - loss:Epoch 00002: val_loss did not improve

Epoch 00002: reducing learning rate to 4.50000006822e-05.
8981/8981 [==============================] - 10s - loss: 0.2906 - acc: 0.8654 - val_loss: 0.2602 - val_acc: 0.8772
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2799 - acc: 0.8721 - val_loss: 0.2459 - val_acc: 0.8798
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2822 - acc: 0.8668 - val_loss: 0.2458 - val_acc: 0.8789
Epoch 6/200
8981/8981 [==============================] - 10s - loss: 0.2771 - acc: 0.8712 - val_loss: 0.2435 - v

In [14]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2814 - acc: 0.8684 - val_loss: 0.2523 - val_acc: 0.8870
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2805 - acc: 0.8712 - val_loss: 0.2446 - val_acc: 0.8870
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2811 - acc: 0.8716 - val_loss: 0.2357 - val_acc: 0.8874
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2760 - acc: 0.8691 - val_loss: 0.2462 - val_acc: 0.8825
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2741 - acc: 0.8746 - val_loss: 0.2342 - val_acc: 0.8905
Epoch 6/200
8981/8981 [==============================] - 10s - loss: 0.2747 - acc: 0.8760 - val_loss: 0.2385 - val_acc: 0.8843
Epoch 7/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2751 - acc: 0.8733Epoch 00006: val_loss did not improve

Epoch 00006: reducing learning rate to 4.99999987369e-05.


In [15]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1.1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2721 - acc: 0.8763 - val_loss: 0.3055 - val_acc: 0.8460
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2743 - acc: 0.8760 - val_loss: 0.2465 - val_acc: 0.8856
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2745 - acc: 0.8800 - val_loss: 0.2397 - val_acc: 0.8803
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2724 - acc: 0.8738 - val_loss: 0.2621 - val_acc: 0.8589
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2659 - acc: 0.8795 - val_loss: 0.2367 - val_acc: 0.8847
Epoch 6/200
8981/8981 [==============================] - 10s - loss: 0.2643 - acc: 0.8816 - val_loss: 0.2494 - val_acc: 0.8741
Epoch 7/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2638 - acc: 0.8789- ETA: 0s - loss: 0.2636 - acc: 0.Epoch 00006: val_loss did not improve

Epoch 00006: reducing le

In [16]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2602 - acc: 0.8835 - val_loss: 0.2351 - val_acc: 0.8896
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2551 - acc: 0.8915 - val_loss: 0.2439 - val_acc: 0.8741
Epoch 3/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2566 - acc: 0.8868Epoch 00002: val_loss did not improve

Epoch 00002: reducing learning rate to 4.99999987369e-05.
8981/8981 [==============================] - 10s - loss: 0.2564 - acc: 0.8869 - val_loss: 0.2353 - val_acc: 0.8879
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2494 - acc: 0.8918 - val_loss: 0.2295 - val_acc: 0.8941
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2545 - acc: 0.8881 - val_loss: 0.2290 - val_acc: 0.8914
Epoch 6/200
8981/8981 [==============================] - 10s - loss: 0.2455 - acc: 0.8904 - val_loss: 0.2312 - val_acc: 0.8968
Epoch 7/200


In [17]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=2e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2691 - acc: 0.8791 - val_loss: 0.2328 - val_acc: 0.8865
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2685 - acc: 0.8732 - val_loss: 0.2395 - val_acc: 0.8887
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2611 - acc: 0.8823 - val_loss: 0.2292 - val_acc: 0.8954
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2603 - acc: 0.8813 - val_loss: 0.2397 - val_acc: 0.8830
Epoch 5/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2555 - acc: 0.8884- ETA: 4s - los - Epoch 00004: val_loss did not improve

Epoch 00004: reducing learning rate to 9.99999974738e-05.
8981/8981 [==============================] - 10s - loss: 0.2547 - acc: 0.8889 - val_loss: 0.2396 - val_acc: 0.8976
Epoch 6/200
8981/8981 [==============================] - 10s - loss: 0.2438 - acc: 0.8918 - val_loss: 0.2266 - val_acc: 0

In [18]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2507 - acc: 0.8869 - val_loss: 0.2334 - val_acc: 0.8870
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2412 - acc: 0.8961 - val_loss: 0.2219 - val_acc: 0.8936
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2401 - acc: 0.8953 - val_loss: 0.2314 - val_acc: 0.8887
Epoch 4/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2386 - acc: 0.8928Epoch 00003: val_loss did not improve

Epoch 00003: reducing learning rate to 4.99999987369e-05.
8981/8981 [==============================] - 10s - loss: 0.2388 - acc: 0.8927 - val_loss: 0.2401 - val_acc: 0.8972
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2363 - acc: 0.8996 - val_loss: 0.2208 - val_acc: 0.9034
Epoch 6/200
8981/8981 [==============================] - 10s - loss: 0.2343 - acc: 0.8996 - val_loss: 0.2311 - val_acc: 0.8985
Epoch 7/200


In [19]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2348 - acc: 0.8975 - val_loss: 0.2548 - val_acc: 0.8652
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2363 - acc: 0.8953 - val_loss: 0.2379 - val_acc: 0.8985
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2373 - acc: 0.8936 - val_loss: 0.2403 - val_acc: 0.8812
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2398 - acc: 0.8992 - val_loss: 0.2282 - val_acc: 0.8865
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2358 - acc: 0.9019 - val_loss: 0.2294 - val_acc: 0.9016
Epoch 6/200
8981/8981 [==============================] - 10s - loss: 0.2310 - acc: 0.8993 - val_loss: 0.2278 - val_acc: 0.8945
Epoch 7/200
8981/8981 [==============================] - 10s - loss: 0.2326 - acc: 0.9018 - val_loss: 0.2294 - val_acc: 0.8950
Epoch 8/200
8981/8981 [==============================] - 10s - 

In [20]:
# Load the model
model = load_model(MODEL_PATH)

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2301 - acc: 0.8995 - val_loss: 0.2227 - val_acc: 0.8968
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2246 - acc: 0.9040 - val_loss: 0.2235 - val_acc: 0.8976
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2264 - acc: 0.8997 - val_loss: 0.2218 - val_acc: 0.8963
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2304 - acc: 0.9066 - val_loss: 0.2368 - val_acc: 0.9039
Epoch 5/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2251 - acc: 0.9011- ETA: 4s - loss: 0.2208 - acc: 0. - ETA - ETA: 1s - Epoch 00004: val_loss did not improve

Epoch 00004: reducing learning rate to 4.99999987369e-05.
8981/8981 [==============================] - 10s - loss: 0.2248 - acc: 0.9015 - val_loss: 0.2349 - val_acc: 0.9008
Epoch 6/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2201 - acc:

In [21]:
# Load the model
model = load_model('./models/model3 (copy).h5')

# Compile
loss = losses.binary_crossentropy
optimizer = optimizers.SGD(lr=1e-4)
metrics = ['accuracy']
model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

# Fit
m_q = 'val_loss'
check_pt = callbacks.ModelCheckpoint(filepath=MODEL_PATH, monitor=m_q, save_best_only=True, verbose=1)
early_stop = callbacks.EarlyStopping(patience=3, monitor=m_q, verbose=1)
reduce_lr = callbacks.ReduceLROnPlateau(patience=1, factor=0.5, monitor=m_q, verbose=1)
callback_list = [check_pt, early_stop, reduce_lr]

model.fit(X_train, y_train, validation_data=(X_val, y_val), callbacks=callback_list, epochs=200)

Train on 8981 samples, validate on 2247 samples
Epoch 1/200
8981/8981 [==============================] - 10s - loss: 0.2392 - acc: 0.8947 - val_loss: 0.2292 - val_acc: 0.8905
Epoch 2/200
8981/8981 [==============================] - 10s - loss: 0.2398 - acc: 0.8934 - val_loss: 0.2256 - val_acc: 0.8959
Epoch 3/200
8981/8981 [==============================] - 10s - loss: 0.2350 - acc: 0.8970 - val_loss: 0.2244 - val_acc: 0.8919
Epoch 4/200
8981/8981 [==============================] - 10s - loss: 0.2424 - acc: 0.8959 - val_loss: 0.2209 - val_acc: 0.9030
Epoch 5/200
8981/8981 [==============================] - 10s - loss: 0.2383 - acc: 0.8976 - val_loss: 0.2278 - val_acc: 0.8976
Epoch 6/200
8928/8981 [============================>.] - ETA: 0s - loss: 0.2349 - acc: 0.8983- ETA: 4s - l - EEpoch 00005: val_loss did not improve

Epoch 00005: reducing learning rate to 4.99999987369e-05.
8981/8981 [==============================] - 10s - loss: 0.2351 - acc: 0.8981 - val_loss: 0.2290 - val_acc: 0.

# Predict Test

In [26]:
# Load test data
df = pd.read_json('./data/test.json')
X_test, y_test = [], []
for im_band1, im_band2 in zip(df['band_1'], df['band_2']):
    im_band1 = np.array(im_band1).reshape(75, 75, 1)
    im_band2 = np.array(im_band2).reshape(75, 75, 1)  
    im_bands_avg = (im_band1 + im_band2) / 2.0
    # Preprocess
    im_band1 /= np.std(im_band1)
    im_band2 /= np.std(im_band2)  
    im_bands_avg /= np.std(im_bands_avg)
    im = np.concatenate([im_band1, im_band2, im_bands_avg], axis=2)
    X_test.append(im)    
X_test = np.array(X_test)
print 'X_test.shape:', X_test.shape

X_test.shape: (8424, 75, 75, 3)


In [27]:
# Load the model
MODEL_PATH = './models/model3 (copy).h5'
model = load_model(MODEL_PATH)

In [28]:
# predict - tta
y_test_p = 0
for func in aug_funcs:
    y_test_p += model.predict(func(X_test), verbose=1).flatten()
y_test_p = y_test_p / len(aug_funcs)

8352/8424 [============================>.] - ETA: 0s

In [29]:
df_sub = pd.DataFrame()
df_sub['id'] = df['id']
df_sub['is_iceberg'] = y_test_p.flatten()
df_sub.to_csv('./submissions/sub9.csv', index=False)